In [1]:
import csv
import math

import numpy as np

### Tensorflow
import tensorflow as tf
from tensorflow.contrib import slim

In [2]:
INPUT_SIZE = 28
SPLIT_SIZE = 4

### train_x = train_x / NORMALIZE
NORMALIZE = 1.0
# NORMALIZE = 255   

In [3]:
### Dataset class
class Dataset(object) :
    
    def __init__(self, path, batch_size, shuffle = True, normalize = 255.0) :
        self.batch_size = batch_size
        self.shuffle    = shuffle
        self.normalize  = normalize
        self.xs, self.ys = self._load_data(path)
        
        self.data_size = self.xs.shape[0]
        self.batch_max = int(math.ceil(self.data_size / float(self.batch_size)))
        
    def _load_data(self, path) :
        reader = csv.reader(open(path), lineterminator = "\n")
        data   = []
        labels = []
        for row in reader :
            tmp = np.asarray([float(x) for x in row])
            
            ### Normalize
            tmp = tmp / self.normalize
            
            data.append(tmp[:INPUT_SIZE*INPUT_SIZE])
            labels.append(tmp[INPUT_SIZE*INPUT_SIZE:])
        
        data   = np.asarray(data)
        labels = np.asarray(labels)
        return (data, labels)

    def __call__(self) :
        indexes = None
        if self.shuffle :
            indexes = np.random.permutation(self.data_size)
        else :
            indexes = np.arange(self.data_size)

        for d_idx in range(self.batch_max) :
            start =     (d_idx + 0) * self.batch_size
            end   = min((d_idx + 1) * self.batch_size, self.data_size)
            
            batch_data_size = len(indexes[start:end])
            xs = np.zeros((batch_data_size, self.xs.shape[1]), dtype = np.float32)
            ys = np.zeros((batch_data_size, self.ys.shape[1]), dtype = np.float32)
            
            for b_idx, r_idx in enumerate(indexes[start:end]) :
                xs[b_idx] = self.xs[r_idx]
                ys[b_idx] = self.ys[r_idx]

            yield (xs, ys)

In [4]:
### Load dataset
dataset = Dataset("training_data_4x4.csv", 10, shuffle = True, normalize = NORMALIZE)

In [5]:
valid_dataset = Dataset("training_data_4x4.csv", 10, shuffle = False, normalize = NORMALIZE)

In [6]:
### Load batch data sample code
# for batch_num, (train_x, train_y) in enumerate(dataset(), 1) :
#     print("Batch {0}".format(batch_num))
#     print("Train shape : {0}".format(train_x.shape))

In [7]:
### for jupyter, interactive session.
sess = tf.InteractiveSession()

In [13]:
### Network settings
NN_INPUT_SIZE  = INPUT_SIZE * INPUT_SIZE
NN_OUTPUT_SIZE = 4 * SPLIT_SIZE

### Input and Output
x  = tf.placeholder(tf.float32, shape=[None, NN_INPUT_SIZE], name = 'data')
y_ = tf.placeholder(tf.float32, shape=[None, NN_OUTPUT_SIZE], name = 'label')
is_training = tf.placeholder(tf.bool, [])

def print_shape(layer) :
    print("Layer \"{0}\" shape : {1}".format(layer.name, layer.shape))

with slim.arg_scope([slim.conv2d, slim.fully_connected], 
                    activation_fn=tf.nn.relu, 
                    weights_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                    biases_initializer=tf.constant_initializer(0.1),
                    normalizer_fn = slim.batch_norm,
                    normalizer_params = {'is_training': is_training},
                   ), \
    slim.arg_scope([slim.max_pool2d], padding='SAME'), \
    slim.arg_scope([slim.conv2d_transpose], 
                   padding='SAME', 
                   activation_fn = None, 
                   stride = 2, 
                   weights_initializer = tf.contrib.layers.xavier_initializer(),
                   normalizer_fn = slim.batch_norm,
                   normalizer_params = {'is_training': is_training}
                  ) :
        
        x_image = slim.array_ops.reshape(x, [-1, 28, 28, 1])
        print_shape(x_image)
        
        ### Convolution
        h_conv1 = slim.conv2d(x_image, 32, [3, 3])
        h_pool1 = slim.max_pool2d(h_conv1, [2, 2])
        print_shape(h_pool1)

        h_conv2 = slim.conv2d(h_pool1, 64, [3, 3])
        h_pool2 = slim.max_pool2d(h_conv2, [2, 2])
        print_shape(h_pool2)

        h_conv3 = slim.conv2d(h_pool2, 64, [3, 3])
        h_pool3 = slim.max_pool2d(h_conv3, [2, 2])
        print_shape(h_pool3)  

        ### Global (fully connect)
        h_pool3_flat = slim.flatten(h_pool3)
        print_shape(h_pool3_flat)
        h_pool3_fc   = slim.fully_connected(h_pool3_flat, NN_OUTPUT_SIZE, activation_fn = None, normalizer_fn = None)
        h_pool3_reconstract = slim.array_ops.reshape(h_pool3_fc, [-1, 4, SPLIT_SIZE, 1])
        print_shape(h_pool3_reconstract)
        
        ### Local (Deconvolution)
        # h_deconv1 = slim.conv2d_transpose(h_pool3,   64, [3, 3])
        # print_shape(h_deconv1)
        # h_deconv2 = slim.conv2d_transpose(h_deconv1,  1, [3, 3], normalizer_fn = None)
        # print_shape(h_deconv2)
        # h_pool3_conv = slim.conv2d(h_pool3, 1, [3, 3], activation_fn = None, normalizer_fn = None)
        h_pool3_conv = slim.conv2d(h_pool3, 1, [3, 3], activation_fn = None, normalizer_fn = None)
        print_shape(h_pool3_conv)
        
        h_pool3_deconv = slim.conv2d_transpose(h_pool3, 1, [1, 1], activation_fn = None, normalizer_fn = None)
        print_shape(h_pool3_deconv)
        
        ### Add global and local
        # h_add = tf.add(h_pool3_reconstract, h_deconv2)
        h_add = tf.add(h_pool3_reconstract, h_pool3_conv)
        y_flat = slim.flatten(h_add)
        print_shape(y_flat)

Layer "Reshape_14:0" shape : (?, 28, 28, 1)
Layer "MaxPool2D_21/MaxPool:0" shape : (?, 14, 14, 32)
Layer "MaxPool2D_22/MaxPool:0" shape : (?, 7, 7, 64)
Layer "MaxPool2D_23/MaxPool:0" shape : (?, 4, 4, 64)
Layer "Flatten_13/Reshape:0" shape : (?, 1024)
Layer "Reshape_15:0" shape : (?, 4, 4, 1)
Layer "Conv_32/BiasAdd:0" shape : (?, 4, 4, 1)
Layer "Conv2d_transpose_6/BiasAdd:0" shape : (?, 8, 8, 1)
Layer "Flatten_14/Reshape:0" shape : (?, 16)


In [44]:
### Loss
loss = tf.losses.mean_squared_error(y_, y_flat)

In [45]:
### Training step (Optimizer)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [46]:
### Initialize session
sess.run(tf.global_variables_initializer())

In [47]:
### Training
EPOCH = 10000
VALIDATION_EPOCH = 100
for epoch in range(1, EPOCH + 1) :
    
    train_loss  = 0
    train_count = 0
    for train_x, train_y in dataset() :
        train_count += 1
        _, tmp_loss = sess.run([train_step, loss], feed_dict = {x: train_x, y_: train_y, is_training: True})
        batch_loss = tmp_loss / train_x.shape[0]
        train_loss += batch_loss
    if train_count > 0 :
        train_loss /= train_count

    ### Validation
    if epoch % VALIDATION_EPOCH == 0 :
        valid_loss  = 0
        valid_count = 0
        for (valid_x, valid_y) in valid_dataset() :
            valid_count += 1
            tmp_loss   = sess.run(loss, feed_dict = {x: valid_x, y_: valid_y, is_training: True}) # 本当は is_training = False!!
            batch_loss = tmp_loss / valid_x.shape[0]
            valid_loss += batch_loss
        if valid_count > 0 :
            valid_loss /= valid_count
        print("Epoch {0} train loss = {1}, valid loss = {2}, sqrt(valid loss) * {4} = {3}".format(epoch, train_loss, valid_loss, math.sqrt(valid_loss) * NORMALIZE, NORMALIZE))

Epoch 100 train loss = 728.0102120535713, valid loss = 793.304541015625, sqrt(valid loss) * 1.0 = 28.165662445886568
Epoch 200 train loss = 539.0798723493303, valid loss = 560.696271623884, sqrt(valid loss) * 1.0 = 23.679025985540115
Epoch 300 train loss = 403.5551339285715, valid loss = 431.5223458426339, sqrt(valid loss) * 1.0 = 20.773115939661867
Epoch 400 train loss = 297.9582013811384, valid loss = 331.25888671875003, sqrt(valid loss) * 1.0 = 18.200518858503735
Epoch 500 train loss = 218.25235770089287, valid loss = 253.10111781529014, sqrt(valid loss) * 1.0 = 15.909152014337224
Epoch 600 train loss = 170.93891601562498, valid loss = 190.23765869140627, sqrt(valid loss) * 1.0 = 13.79266684479134
Epoch 700 train loss = 118.62942417689733, valid loss = 143.6462637765067, sqrt(valid loss) * 1.0 = 11.985251927953232
Epoch 800 train loss = 91.78487766810827, valid loss = 106.05640171595982, sqrt(valid loss) * 1.0 = 10.298368886185804
Epoch 900 train loss = 64.80100446428571, valid loss

Epoch 7000 train loss = 0.3995631166866847, valid loss = 1.104120741571699, sqrt(valid loss) * 1.0 = 1.050771498267677
Epoch 7100 train loss = 0.3304118905748639, valid loss = 1.2163497516087123, sqrt(valid loss) * 1.0 = 1.1028824740690697
Epoch 7200 train loss = 0.4726739849363054, valid loss = 1.2196226716041565, sqrt(valid loss) * 1.0 = 1.1043652799704256
Epoch 7300 train loss = 0.3109609433582851, valid loss = 1.0693596363067628, sqrt(valid loss) * 1.0 = 1.034098465479358
Epoch 7400 train loss = 0.35304806402751376, valid loss = 0.9392268589564733, sqrt(valid loss) * 1.0 = 0.9691371724149649
Epoch 7500 train loss = 0.37337128434862404, valid loss = 1.0170026847294398, sqrt(valid loss) * 1.0 = 1.008465509935486
Epoch 7600 train loss = 0.3360155020441328, valid loss = 1.017811301776341, sqrt(valid loss) * 1.0 = 1.0088663448526476
Epoch 7700 train loss = 0.2534714494432722, valid loss = 1.1379922253744943, sqrt(valid loss) * 1.0 = 1.0667671842414794
Epoch 7800 train loss = 0.398817556

In [31]:
### Save session
saver = tf.train.Saver()
saver.save(sess, "cae_model.ckpt")

'cae_model.ckpt'

In [32]:
### Load session
saver.restore(sess, "cae_model.ckpt")

INFO:tensorflow:Restoring parameters from cae_model.ckpt
